In [33]:
# 운영 DB 적재에 해당된다

In [34]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [35]:
# 데이터마트용 계정 생성
# mysql> create user bigSVC@'%' identified by 'bigSVC1234@';
# Query OK, 0 rows affected (0.01 sec)

# mysql> grant all privileges on *.* to bigSVC@'%' with grant option;
# - with grant option : 해당 계정이 받은 권한을 다른 계쩡에게 전달해주는게 가능하게 하겠다
# Query OK, 0 rows affected (0.02 sec)


In [36]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 


In [37]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

### JDBC 연결정보

In [38]:
# DM DB 연결정보
conf_dm = {
      'url':'jdbc:mysql://localhost:3306/etlMysqlDM?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}

# service DB 연결정보
conf_svc = {
    'url': 'jdbc:mysql://127.0.0.1:3306/etlMysqlSVC?characterEncoding=utf8&serverTimezone=Asia/Seoul',
    'props': {
        'user': 'bigSVC',
        'password': 'bigSVCl1234@'
    }
}


## Load DataMart

### 1. 지역별 단위면적(km)당 인구수와 코로나 확진자 수 데이터 저장
- base data load
    - LOC table : 지역별 인구와 면적
    - dm db data read -> 처리코드 -> service db dataload

In [30]:
spark.read.jdbc(url=conf_dm["url"], table = "LOC", properties=conf_dm["props"]).show()

+----+------+---------+
| LOC|  AREA|     POPU|
+----+------+---------+
|서울|  605 | 9736027 |
|부산|  770 | 3396109 |
|대구|  883 | 2412642 |
|인천| 1065 | 3014739 |
|광주|  501 | 1462545 |
|대전|  540 | 1469543 |
|울산| 1062 | 1138419 |
|세종|  465 |  376779 |
|경기|10195 |13925862 |
|강원|16830 | 1555876 |
|충북| 7407 | 1633472 |
|충남| 8246 | 2181835 |
|전북| 8070 | 1817186 |
|전남|12348 | 1865459 |
|경북|19034 | 2677709 |
|경남|10541 | 3377331 |
|제주| 1850 |  697476 |
+----+------+---------+



In [31]:
# db table 읽기, 저장함수
# read 함수
def find_data(config, table_name) :
    return spark.read.jdbc(url=config["url"], table = table_name, properties=config["props"])

# write 함수
def save_data(config, df, table_name) :
    df.write.jdbc(url=config["url"], table = table_name, mode="append", properties=config["props"])

#### 단위 면적당 인구수, 코로나 확진자 수 현황
- DM DB의 LOC, CORPNA_PATIENTS 테이블

In [21]:
popu = find_data(conf_dm, "LOC")
patients = find_data(conf_dm, "CORONA_PATIENTS")

In [22]:
popu.columns
patients.columns

['LOC', 'AREA', 'POPU']

['LOC', 'DEATH_CNT', 'DEF_CNT', 'LOC_OCC_CNT', 'QUR_RATE', 'STD_DAY']

['LOC', 'DEATH_CNT', 'DEF_CNT', 'LOC_OCC_CNT', 'QUR_RATE', 'STD_DAY']   
 지역   누적사망자수  누적 발생수  지역발생수     만명당 발생률  기준일자

In [23]:
popu.count()
patients.count()

17

119

In [24]:
popu.join(patients, on="LOC").tail(10)

[Row(LOC='충남', AREA='8246 ', POPU='2181835 ', DEATH_CNT='165', DEF_CNT='19553', LOC_OCC_CNT='126', QUR_RATE='923', STD_DAY='2022-01-13'),
 Row(LOC='충남', AREA='8246 ', POPU='2181835 ', DEATH_CNT='163', DEF_CNT='19422', LOC_OCC_CNT='112', QUR_RATE='916', STD_DAY='2022-01-12'),
 Row(LOC='충남', AREA='8246 ', POPU='2181835 ', DEATH_CNT='165', DEF_CNT='19659', LOC_OCC_CNT='99', QUR_RATE='928', STD_DAY='2022-01-14'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='121', DEF_CNT='12455', LOC_OCC_CNT='37', QUR_RATE='780', STD_DAY='2022-01-16'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='121', DEF_CNT='12497', LOC_OCC_CNT='38', QUR_RATE='782', STD_DAY='2022-01-17'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='120', DEF_CNT='12416', LOC_OCC_CNT='77', QUR_RATE='777', STD_DAY='2022-01-15'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='123', DEF_CNT='12555', LOC_OCC_CNT='55', QUR_RATE='786', STD_DAY='2022-01-18'),
 Row(LOC='충북', AREA='7407 ', POPU='1633

In [25]:
popPatient = popu.join(patients, on="LOC") \
                 .select("LOC",
                        ceil(col("POPU")/col("AREA")).alias("POPU_DENST"),
                        "QUR_RATE", "STD_DAY") \
                 .orderBy(col("STD_DAY"))

In [26]:
popPatient.show(5)
popPatient.tail(5)

+----+----------+--------+----------+
| LOC|POPU_DENST|QUR_RATE|   STD_DAY|
+----+----------+--------+----------+
|광주|      2920|     642|2022-01-12|
|세종|       811|     621|2022-01-12|
|대구|      2733|    1019|2022-01-12|
|강원|        93|     863|2022-01-12|
|대전|      2722|     884|2022-01-12|
+----+----------+--------+----------+
only showing top 5 rows



[Row(LOC='전남', POPU_DENST=152, QUR_RATE='433', STD_DAY='2022-01-18'),
 Row(LOC='전북', POPU_DENST=226, QUR_RATE='629', STD_DAY='2022-01-18'),
 Row(LOC='제주', POPU_DENST=378, QUR_RATE='724', STD_DAY='2022-01-18'),
 Row(LOC='충남', POPU_DENST=265, QUR_RATE='950', STD_DAY='2022-01-18'),
 Row(LOC='충북', POPU_DENST=221, QUR_RATE='786', STD_DAY='2022-01-18')]

- 각 DB에 고유번호를 추가해서 레코드를 유일하게 구별하는 컬럼으로 사용
    - spark 데이터 프레임의 statfuctions 패키지 함수 중 monotonically_increasing_id
    - 데이터프레임의 로우에 매핑되는 고유 id 반환

In [27]:
popPatFin = popPatient.withColumn("CP_IDX", monotonically_increasing_id())
popPatFin.show()

+----+----------+--------+----------+------+
| LOC|POPU_DENST|QUR_RATE|   STD_DAY|CP_IDX|
+----+----------+--------+----------+------+
|강원|        93|     863|2022-01-12|     0|
|경기|      1366|    1483|2022-01-12|     1|
|경남|       321|     665|2022-01-12|     2|
|경북|       141|     622|2022-01-12|     3|
|광주|      2920|     642|2022-01-12|     4|
|대구|      2733|    1019|2022-01-12|     5|
|대전|      2722|     884|2022-01-12|     6|
|부산|      4411|     815|2022-01-12|     7|
|서울|     16093|    2503|2022-01-12|     8|
|세종|       811|     621|2022-01-12|     9|
|울산|      1072|     651|2022-01-12|    10|
|인천|      2831|    1330|2022-01-12|    11|
|전남|       152|     384|2022-01-12|    12|
|전북|       226|     589|2022-01-12|    13|
|제주|       378|     717|2022-01-12|    14|
|충남|       265|     916|2022-01-12|    15|
|충북|       221|     764|2022-01-12|    16|
|강원|        93|     869|2022-01-13|    17|
|경기|      1366|    1495|2022-01-13|    18|
|경남|       321|     668|2022-01-13|    19|
+----

In [39]:
save_data(conf_svc, popPatFin, "CO_POP_DENSITY")

### 2. 백신접종 완료자와 코로나 발생현황 데이터 저장
- 백신 접종 3차 접종자수(누적) : 인구 10만명당 접종자수
- 코로나 발생현황 : QUR_RATE(인구 10만명당 발생자수)

In [40]:
vaccine = find_data(conf_dm, "CORONA_VACCINE")
vaccine.show(3)
vaccine.printSchema()

+----+----------+-------+---------+
| loc|   std_day|level_2|    V_CNT|
+----+----------+-------+---------+
|경남|2022-01-18|     v2|2783896.0|
|경남|2022-01-18|     v1|  2848083|
|경남|2022-01-18|     v3|  22358.0|
+----+----------+-------+---------+
only showing top 3 rows

root
 |-- loc: string (nullable = true)
 |-- std_day: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- V_CNT: string (nullable = true)



In [41]:
vaccine.withColumn("V_CNT", vaccine["V_CNT"].cast(IntegerType()))
vaccine.show(2)

DataFrame[loc: string, std_day: string, level_2: string, V_CNT: int]

+----+----------+-------+---------+
| loc|   std_day|level_2|    V_CNT|
+----+----------+-------+---------+
|경남|2022-01-18|     v2|2783896.0|
|경남|2022-01-18|     v1|  2848083|
+----+----------+-------+---------+
only showing top 2 rows



In [61]:
# long df --> wide df 변환 (V_TH 컬럼 값을 컬럼으로 변환)
# pivot 사용 : pandas df로 변환
pdVac = vaccine.pandas_api()
type(pdVac)

pyspark.pandas.frame.DataFrame

In [62]:
pdVac = pdVac.rename(columns={"loc":"LOC","std_day":"STD_DAY", "level_2":"V_TH"})
pdVac["V_CNT"] = pdVac["V_CNT"].astype(int)

In [63]:
pdVacV1 = pdVac.pivot_table(index=['LOC','STD_DAY'], columns='V_TH', values='V_CNT')
pdVacV1.head(1)
pdVacV1 = pdVacV1.reset_index()
pdVacV1.head(1)

,V_TH,v1,v2,v3
LOC,STD_DAY,,,
전남,2022-01-15,1631047.0,1599517.0,20760.0


V_TH,LOC,STD_DAY,v1,v2,v3
0,전남,2022-01-15,1631047.0,1599517.0,20760.0


- Spark 데이터 프레임의 StatFunctions 패키지 함수 중 monotonically_increasing_id
    - 데이터 프레임의 로우에 되는매핑되는  고유 id 반환



In [65]:
vaccine = pdVacV1.to_spark()
vaccine.show(2)

+----+----------+---------+---------+-------+
| LOC|   STD_DAY|       v1|       v2|     v3|
+----+----------+---------+---------+-------+
|전남|2022-01-15|1631047.0|1599517.0|20760.0|
|울산|2022-01-17| 956456.0| 930969.0|  240.0|
+----+----------+---------+---------+-------+
only showing top 2 rows



In [66]:
# 지역별 인구 10만명당 백신 접종률 계산
# 인구 data 필요
popu = find_data(conf_dm, 'LOC')
# 코로나 현황 data 필요
patients = find_data(conf_dm,'CORONA_PATIENTS')

In [68]:
vacRate = vaccine.join(popu, on="LOC") \
                 .select("LOC",
                         "STD_DAY",
                         ceil(col("v3")/col("POPU")*100000).alias("THRD_RATE"))
vacRate.show(1)

+----+----------+---------+
| LOC|   STD_DAY|THRD_RATE|
+----+----------+---------+
|경북|2022-01-16|      450|
+----+----------+---------+
only showing top 1 row



In [69]:
coPatVac = vacRate.join(patients, on = ["LOC", "STD_DAY"]) \
                  .select("LOC", "STD_DAY", "THRD_RATE", "QUR_RATE")

In [70]:
coPatVac.show(3)

+----+----------+---------+--------+
| LOC|   STD_DAY|THRD_RATE|QUR_RATE|
+----+----------+---------+--------+
|경북|2022-01-16|      450|     637|
|경북|2022-01-15|      967|     633|
|경북|2022-01-17|       13|     641|
+----+----------+---------+--------+
only showing top 3 rows



In [71]:
coPatVac = coPatVac.withColumn("CV_IDX", monotonically_increasing_id())
coPatVac.show(2)

+----+----------+---------+--------+------+
| LOC|   STD_DAY|THRD_RATE|QUR_RATE|CV_IDX|
+----+----------+---------+--------+------+
|경북|2022-01-16|      450|     637|     0|
|경북|2022-01-15|      967|     633|     1|
+----+----------+---------+--------+------+
only showing top 2 rows



In [72]:
save_data(conf_svc, coPatVac, "CO_VACCINE_PATIENT")

### 3. 다중이용시설과 코로나 확진자 수의 상관관계
- DM DB LOC_FACILITY_CNT
    - 인구 10만명당 다중 이용시설 수
- DM DB CORONA_PATIENTS

In [73]:
facil = find_data(conf_dm, "LOC_FACILITY_CNT")
facil.show(1)

+----+-------+
| LOC|FAC_CNT|
+----+-------+
|경북|    944|
+----+-------+
only showing top 1 row



In [75]:
facPop = popu.join(facil, on = "LOC") \
             .select("LOC", 
                     ceil(facil.FAC_CNT/popu.POPU*100000).alias("FAC_POPU"))
facPop.show(1)

+----+--------+
| LOC|FAC_POPU|
+----+--------+
|강원|      38|
+----+--------+
only showing top 1 row



In [76]:
coFacPat = patients.join(facPop, on="LOC") \
                   .select("LOC", "FAC_POPU", "QUR_RATE", "STD_DAY") \
                   .withColumn("CF_IDX", monotonically_increasing_id())
coFacPat.show(2)

+----+--------+--------+----------+------+
| LOC|FAC_POPU|QUR_RATE|   STD_DAY|CF_IDX|
+----+--------+--------+----------+------+
|경북|      36|     633|2022-01-15|     0|
|경북|      36|     641|2022-01-17|     1|
+----+--------+--------+----------+------+
only showing top 2 rows



In [77]:
save_data(conf_svc, coFacPat, "CO_FACT_PATIENTS")

### 4. 요일별 코로나 확진자 수를 구해보자
- 코로나 현황 data : STD_DAY 기준일 data -> 요일로 변경(dayofweek()) -> 요일별 그룹 생성후 신규 확진자수 합산

In [78]:
# 코로나 현황 data 필요
patients = find_data(conf_dm, "CORONA_PATIENTS")

# 기준일을 요일로 변환
patWeek = patients.withColumn("DAY_OF_WEEK", dayofweek(col("STD_DAY")))
patWeek.show(5)

+----+---------+-------+-----------+--------+----------+-----------+
| LOC|DEATH_CNT|DEF_CNT|LOC_OCC_CNT|QUR_RATE|   STD_DAY|DAY_OF_WEEK|
+----+---------+-------+-----------+--------+----------+-----------+
|강원|      110|  13838|         94|     899|2022-01-18|          3|
|경남|      113|  22394|        108|     676|2022-01-15|          7|
|서울|     2033| 240862|        823|    2533|2022-01-15|          7|
|서울|     2063| 243337|        684|    2559|2022-01-18|          3|
|대전|      187|  13262|        101|     913|2022-01-18|          3|
+----+---------+-------+-----------+--------+----------+-----------+
only showing top 5 rows



In [79]:
weekUp = patWeek.groupby(patWeek.DAY_OF_WEEK).agg(sum(col("LOC_OCC_CNT")).alias("PATIENTS"))
weekUp.show()

+-----------+--------+
|DAY_OF_WEEK|PATIENTS|
+-----------+--------+
|          1|  3813.0|
|          6|  4133.0|
|          3|  3763.0|
|          5|  3776.0|
|          4|  4007.0|
|          7|  4077.0|
|          2|  3551.0|
+-----------+--------+



In [80]:
weekUp = weekUp.withColumn("DAY_OF_WEEK", when(weekUp.DAY_OF_WEEK == 1, "MON")
                                        .when(weekUp.DAY_OF_WEEK == 2, "TUE")
                                        .when(weekUp.DAY_OF_WEEK == 3, "WED")
                                        .when(weekUp.DAY_OF_WEEK == 4, "THE")
                                        .when(weekUp.DAY_OF_WEEK == 5, "FRI")
                                        .when(weekUp.DAY_OF_WEEK == 6, "SAT")
                                        .when(weekUp.DAY_OF_WEEK == 7, "SUN")
                 )
weekUp.show()

+-----------+--------+
|DAY_OF_WEEK|PATIENTS|
+-----------+--------+
|        MON|  3813.0|
|        SAT|  4133.0|
|        WED|  3763.0|
|        FRI|  3776.0|
|        THE|  4007.0|
|        SUN|  4077.0|
|        TUE|  3551.0|
+-----------+--------+



In [81]:
# long -> wide 형으로 변경 후 저장
pdWeek = weekUp.pandas_api()
pdWeek

,DAY_OF_WEEK,PATIENTS
0,MON,3813.0
1,SAT,4133.0
2,WED,3763.0
3,FRI,3776.0
4,THE,4007.0
5,SUN,4077.0
6,TUE,3551.0


In [82]:
pdWeekpvt = pdWeek.pivot_table(columns="DAY_OF_WEEK", values="PATIENTS")
pdWeek = pdWeekpvt.reset_index()

In [83]:
weekUp = pdWeek.to_spark()
weekUp.show()

/root/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+--------+------+------+------+------+------+------+------+
|   index|   FRI|   MON|   SAT|   SUN|   THE|   TUE|   WED|
+--------+------+------+------+------+------+------+------+
|PATIENTS|3776.0|3813.0|4133.0|4077.0|4007.0|3551.0|3763.0|
+--------+------+------+------+------+------+------+------+



In [84]:
weekUp.withColumn("STD_DAY", lit(cal_std_day(365*3+4))).show()

+--------+------+------+------+------+------+------+------+----------+
|   index|   FRI|   MON|   SAT|   SUN|   THE|   TUE|   WED|   STD_DAY|
+--------+------+------+------+------+------+------+------+----------+
|PATIENTS|3776.0|3813.0|4133.0|4077.0|4007.0|3551.0|3763.0|2022-01-18|
+--------+------+------+------+------+------+------+------+----------+



In [85]:
save_data(conf_svc, weekUp, "CO_WEEKDAY")